# Lesson 5: Bandwidth

In [1]:
!pip install -r requirements.txt


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Welcome to Lesson 5!

To access the `requirements.txt` and `utils5.py` file for this course, go to `File` and click `Open`.

#### 1. Load imports

In [2]:
import sys
import os
sys.path.insert(0, os.getcwd())  # Ajoute le dossier courant au chemin Python


In [3]:
from flwr.client.mod import parameters_size_mod

from utils5 import *

#### 2.  Define the model

*  Initialize the model.

In [4]:
model = AutoModelForCausalLM.from_pretrained(
    "EleutherAI/pythia-14m",
    cache_dir="./pythia-14m/cache",
)

c:\Users\Dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connection.py:430: DeprecationWarning: ssl.match_hostname() is deprecated
  match_hostname(cert, asserted_hostname)


Find more information about [EleutherAI/pythia-14m](https://huggingface.co/EleutherAI/pythia-14m)

* Get some Model values.

In [5]:
vals = model.state_dict().values()
total_size_bytes = sum(p.element_size() * p.numel() for p in vals)
total_size_mb = int(total_size_bytes / (1024**2))

log(INFO, "Model size is: {} MB".format(total_size_mb))

INFO : Model size is: 53 MB


* Define the FlowerClient.

In [6]:
class FlowerClient(NumPyClient):
    def __init__(self, net):
        self.net = net

    def fit(self, parameters, config):
        set_weights(self.net, parameters)
        # No actual training here
        return get_weights(self.net), 1, {}

    def evaluate(self, parameters, config):
        set_weights(self.net, parameters)
        # No actual evaluation here
        return float(0), int(1), {"accuracy": 0}


def client_fn(context: Context) -> FlowerClient:
    return FlowerClient(model).to_client()


client = ClientApp(
    client_fn,
    mods=[parameters_size_mod],
)

* Define the custom strategy: BandwidthTrackingFedAvg.

In [7]:
bandwidth_sizes = []


class BandwidthTrackingFedAvg(FedAvg):
    def aggregate_fit(self, server_round, results, failures):
        if not results:
            return None, {}

        # Track sizes of models received
        for _, res in results:
            ndas = parameters_to_ndarrays(res.parameters)
            size = int(sum(n.nbytes for n in ndas) / (1024**2))
            log(INFO, f"Server receiving model size: {size} MB")
            bandwidth_sizes.append(size)

        # Call FedAvg for actual aggregation
        return super().aggregate_fit(server_round, results, failures)

    def configure_fit(self, server_round, parameters, client_manager):
        # Call FedAvg for actual configuration
        instructions = super().configure_fit(
            server_round, parameters, client_manager
        )

        # Track sizes of models to be sent
        for _, ins in instructions:
            ndas = parameters_to_ndarrays(ins.parameters)
            size = int(sum(n.nbytes for n in ndas) / (1024**2))
            log(INFO, f"Server sending model size: {size} MB")
            bandwidth_sizes.append(size)

        return instructions

In [8]:
params = ndarrays_to_parameters(get_weights(model))

def server_fn(context: Context):
    strategy = BandwidthTrackingFedAvg(
        fraction_evaluate=0.0,
        initial_parameters=params,
    )
    config = ServerConfig(num_rounds=1)
    return ServerAppComponents(
        strategy=strategy,
        config=config,
    )


server = ServerApp(server_fn=server_fn)

* Run the simulation.

In [9]:
run_simulation(server_app=server,
               client_app=client,
               num_supernodes=2,
               backend_config=backend_setup
               )

INFO : Starting Flower ServerApp, config: num_rounds=1, no round_timeout
INFO : 
INFO : [INIT]
INFO : Using initial global parameters provided by strategy
INFO : Evaluating initial global parameters
INFO : 
INFO : [ROUND 1]
INFO : Server sending model size: 53 MB
INFO : Server sending model size: 53 MB
INFO : configure_fit: strategy sampled 2 clients (out of 2)
(pid=6884) 2025-05-14 10:43:38.371827: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the e
(pid=6884) n
(pid=6884) v
(pid=6884) i
(pid=21644) 2025-05-14 10:43:38.352888: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
(pid=36648) 2025-05-14 10:43:38.

* Log how much bandwidth was used!

In [10]:
log(INFO, "Total bandwidth used: {} MB".format(sum(bandwidth_sizes)))

INFO : Total bandwidth used: 106 MB
